In [ ]:
# Importing necessary libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
# Load dataset
df = pd.read_csv('../Datasets/CleanData.csv')

In [ ]:
# Extract features and target
X = df.iloc[:, 0:7].values
Y = df.iloc[:, 7].values

In [ ]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=1)

In [ ]:
# Define and evaluate KNN model
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
knn_acc = accuracy_score(y_test, y_pred)
knn_cls = classification_report(y_test, y_pred)
knn_acc
knn_cls

In [ ]:
# Define and evaluate GaussianNB model
gnb = GaussianNB()
gnb.fit(X_train, y_train)
y_pred_gnb = gnb.predict(X_test)
gnb_acc = accuracy_score(y_test, y_pred_gnb)
gnb_cls = classification_report(y_test, y_pred_gnb)
gnb_acc
gnb_cls

In [ ]:
# Define and evaluate DecisionTree model
clf = DecisionTreeClassifier(criterion='gini', max_depth=10)
clf.fit(X_train, y_train)
y_pred_clf = clf.predict(X_test)
dt_acc = accuracy_score(y_test, y_pred_clf)
dt_cls = classification_report(y_test, y_pred_clf)
dt_acc
dt_cls

In [ ]:
# Define and evaluate RandomForest model
rf = RandomForestClassifier(n_estimators=50, random_state=2)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
rf_acc = accuracy_score(y_test, y_pred_rf)
rf_cls = classification_report(y_test, y_pred_rf)
rf_acc
rf_cls

In [ ]:
# Define and evaluate SVM model
svm = SVC(kernel='linear', probability=True)
svm.fit(X_train, y_train)
y_pred_svm = svm.predict(X_test)
svm_acc = accuracy_score(y_test, y_pred_svm)
svm_cls = classification_report(y_test, y_pred_svm)
svm_acc
svm_cls

In [ ]:
# Evaluate models for Depression, Anxiety, and Stress
datasets = {'Depression': '../Datasets/Depression.csv', 'Anxiety': '../Datasets/Anxiety.csv', 'Stress': '../Datasets/Stress.csv'}
models = [knn, gnb, clf, rf, svm]
model_names = ['KNN', 'GaussianNB', 'DecisionTree', 'RandomForest', 'SVM']
for condition, path in datasets.items():
    df_cond = pd.read_csv(path)
    X_cond = df_cond.iloc[:, :-1].values
    Y_cond = df_cond.iloc[:, -1].values
    X_train_cond, X_test_cond, y_train_cond, y_test_cond = train_test_split(X_cond, Y_cond, test_size=0.3, random_state=1)
    print(f'\nCondition: {condition}')
    for model, name in zip(models, model_names):
        model.fit(X_train_cond, y_train_cond)
        y_pred_cond = model.predict(X_test_cond)
        acc_cond = accuracy_score(y_test_cond, y_pred_cond)
        cls_cond = classification_report(y_test_cond, y_pred_cond)
        print(f'\nModel: {name}')
        print(f'Accuracy: {acc_cond}')
        print(f'Classification Report:\n{cls_cond}')